# HMM

In [ ]:
from decimal import Decimal
def Learn(observations,currentIteration,Probabilities,Transitions,B):
    if (currentIteration!= 1):
        N = len(observations)
        MinValue = 0
        currentIteration = 1
    States = 3
    epsilon=[]
    gamma=[]
    for  i in range(0,N-1):
        T = len(observations[i])
        epsilon.append([T, States, States])
        gamma.append([T, States])
    #// Calculate initial model log-likelihood
    oldLikelihood = float(0)
    newLikelihood = float(0)
    for i in range(0,N-1):
        sequence = observations[i]
        T = len(sequence);
        scaling=[];
        fwd = forward(observations[i], scaling,Probabilities,Transitions,B);
        bwd = backward(observations[i], scaling,Probabilities,Transitions,B);
        #// 2nd step - Determining the frequency of the transition-emission pair values
        #//            and dividing it by the probability of the entire string.

        #// Calculate gamma values for next computations
        for t in range(0,T-1):
            s = 0;
            for k in range(0,States-1):
                gamma[i][t, k] = fwd[t, k] * bwd[t, k];
                s += gamma[i][t, k]
            if (s != 0):#// Scaling
                for k in range(0,States-1):
                    gamma[i][t, k] /= s
        #// Calculate epsilon values for next computations
        for t in range(0,T-2):
            s = 0;

            for k in range(0,States-1):
                for l in range(0,States-2):
                    epsilon[i][t, k, l] = fwd[t, k] * A[k, l] * bwd[t + 1, l] * B[l, sequence[t + 1]];
                    s +=epsilon[i][t, k, l]
            if (s != 0): #// Scaling
                for k in range(0,States-1):
                    for l in range(0,States-1):
                        epsilon[i][t, k, l] /= s;

        #// Compute log-likelihood for the given sequence
        for t in range(0,len(scaling)):
            newLikelihood += Math.Log(scaling[t])
            
        #// Average the likelihood for all sequences
        newLikelihood /= observations.Length

In [3]:
def forward(observations, c,Probabilities,Transitions,B):
    T = len(observations)
    pi = Probabilities
    A = Transitions
    States=3
    fwd = [T, States]
    c= [T]


    #// 1. Initialization
    for i in range(0,States-1):
        fwd[0, i] = pi[i] * [i, observations[0]]
        c[0] += fwd[0, i]
        
    if (c[0] != 0): #// Scaling
        for i in range(0,States-1):
            fwd[0, i] = fwd[0, i] / c[0]
    #// 2. Induction
    for t in range(1,T-1):
        for i in range(0,States-1):
            p = B[i, observations[t]]
            sum = 0.0
            for j in range(0,States-1):
                sum += fwd[t - 1, j] * A[j, i]
            fwd[t, i] = sum * p
            c[t] += fwd[t, i] #// scaling coefficient
        if (c[t] != 0): #// Scaling
            for i in range(0,States-1):
                fwd[t, i] = fwd[t, i] / c[t]
    return fwd

def backward(observations, c,Probabilities,Transitions,B):
    T = len(observations)
    pi = Probabilities;
    A = Transitions;
    bwd = [T, States]
    States=3
    #// For backward variables, we use the same scale factors
    #//   for each time t as were used for forward variables.

    #// 1. Initialization
    for i in range(0,States-1):
        bwd[T - 1, i] = 1.0 / c[T - 1]

    #// 2. Induction
    for i in range(T-2,0,-1):
        for i in range(0, States-1):
            sum = 0
            for j in range(0, States-1):
                sum += A[i, j] * [j, observations[t + 1]] * bwd[t + 1, j]
            bwd[t, i] += sum / c[t]
    return bwd

In [4]:
sequences1 = [['A','A','B','B','C','C','D','D'],['A','B','B','C','B','B','D','D'],['A','C','B','C','B','C','D'],['A','D'],['A','C','B','C','B','A','B','C','D','D'],['B','A','B','A','A','D','D','D'],['B','A','B','C','D','C','C'],['A','B','D','B','B','C','C','D','D'],['A','B','A','A','A','C','D','C','C','D'],['A','B','D']]
sequences2= [['D','D','C','C','B','B','A','A'],['D','D','A','B','C','B','A'],['C','D','C','D','C','B','A','B','A'],['D','D','B','B','A'],['D','A','D','A','C','B','B','A','A'],['C','D','D','C','C','B','A'],['B','D','D','B','C','A','A','A','A'],['B','B','A','B','B','D','D','D','C','D'],['D','D','A','D','D','B','C','A','A'],['D','D','C','A','A','A']]

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

states = ['A', 'B', 'C','D']
pi = [0.25, 0.25, 0.25, 0.25]
o_states =['1','2','3']
q_df = pd.DataFrame(columns=states, index=states)
q_df.loc[states[0]] = [0.5, 0.5, 0.0, 0.0]
q_df.loc[states[1]] = [0.0, 0.5, 0.5, 0.0]
q_df.loc[states[2]] = [0.0, 0.0, 0.5, 0.5]
q_df.loc[states[3]] = [0.0, 0.0, 0.0, 1.0]

b_df = pd.DataFrame(columns=states, index=o_states)
b_df.loc[o_states[0]] = [0.25, 0.25, 0.25, 0.25]
b_df.loc[o_states[1]] = [0.25, 0.25, 0.25, 0.25]
b_df.loc[o_states[2]] = [0.25, 0.25, 0.25, 0.25]


Probabilities = np.array(pi, dtype=float)
Transitions = q_df.as_matrix().tolist()
B=b_df.as_matrix().tolist()
state_space = pd.Series(pi, index=states, name='states')
print(state_space)
print(state_space.sum())
print(q_df)

q = q_df.values
print('\n', q, q.shape, '\n')
print(q_df.sum(axis=1))


from pprint import pprint 

def _get_markov_edges(Q):
    edges = {}
    for col in Q.columns:
        for idx in Q.index:
            edges[(idx,col)] = Q.loc[idx,col]
    return edges

edges_wts = _get_markov_edges(q_df)
pprint(edges_wts)
print(b_df)

b = b_df.values
print('\n', b, b.shape, '\n')
print(b_df.sum(axis=1))

emit_edges_wts = _get_markov_edges(b_df)
pprint(emit_edges_wts)

A    0.25
B    0.25
C    0.25
D    0.25
Name: states, dtype: float64
1.0
     A    B    C    D
A  0.5  0.5    0    0
B    0  0.5  0.5    0
C    0    0  0.5  0.5
D    0    0    0    1

 [[0.5 0.5 0.0 0.0]
 [0.0 0.5 0.5 0.0]
 [0.0 0.0 0.5 0.5]
 [0.0 0.0 0.0 1.0]] (4, 4) 

A    1.0
B    1.0
C    1.0
D    1.0
dtype: float64
{('A', 'A'): 0.5,
 ('A', 'B'): 0.5,
 ('A', 'C'): 0.0,
 ('A', 'D'): 0.0,
 ('B', 'A'): 0.0,
 ('B', 'B'): 0.5,
 ('B', 'C'): 0.5,
 ('B', 'D'): 0.0,
 ('C', 'A'): 0.0,
 ('C', 'B'): 0.0,
 ('C', 'C'): 0.5,
 ('C', 'D'): 0.5,
 ('D', 'A'): 0.0,
 ('D', 'B'): 0.0,
 ('D', 'C'): 0.0,
 ('D', 'D'): 1.0}
      A     B     C     D
1  0.25  0.25  0.25  0.25
2  0.25  0.25  0.25  0.25
3  0.25  0.25  0.25  0.25

 [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]] (3, 4) 

1    1.0
2    1.0
3    1.0
dtype: float64
{('1', 'A'): 0.25,
 ('1', 'B'): 0.25,
 ('1', 'C'): 0.25,
 ('1', 'D'): 0.25,
 ('2', 'A'): 0.25,
 ('2', 'B'): 0.25,
 ('2', 'C'): 0.25,
 ('2', 'D'): 0.25,
 ('3', 'A')

In [ ]:
Learn(sequences1,0,Probabilities,Transitions,B)
Learn(sequences2,0,Probabilities,Transitions,B)